In [1]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../')
import gdc19
# Be sure to set your working directory!!!
# this is the path to the Google Drive folder
gdc19.setup_adam()

'/Volumes/GoogleDrive/My Drive/utah-forge'

In [2]:
import vtki
import omfvtk
import PVGeo

-----

# Initial Inspection

This notebook loads up all the data and works through an initial inspection.

In [3]:
proj = omfvtk.load_project(gdc19.get_omf_project_filename())
proj

(MultiBlock)0x10a1056a8

In [4]:
topo = proj['land_surface']
topo

(UnstructuredGrid)0x1297fe7c8

# Integrated Scene

Now use `vtki` to create and integrated rendering of all the available data


In [5]:
p = vtki.BackgroundPlotter()
# p.add_bounds_axes()

In [6]:
p.add_mesh(proj['temperature'], name='temp', 
           cmap='coolwarm', point_size=10, 
           render_points_as_spheres=True, stitle='Temperature')
p.add_mesh(proj['negro_mag_fault'], name='negro_mag_fault',
           show_edges=False, color='blue', opacity=0.5)
p.add_mesh(proj['opal_mound_fault'], name='opal_mound_fault',
           show_edges=False, color='orange', opacity=0.5)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4948

In [7]:
# We can update this cell to change the texture
p.add_mesh(topo, 
           texture='geo_aer', 
           show_edges=False, 
           name='topo', 
           reset_camera=False)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4d08

In [8]:
# add the temperature boundaries
p.add_mesh(proj['temp_175c'], name='temp_175c', 
           show_edges=False, color='aquamarine',
           style='wireframe')
p.add_mesh(proj['temp_225c'], name='temp_225c', 
           show_edges=False, color='firebrick',
           style='wireframe')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4e88

In [9]:
# Add the shape file stuff
p.add_mesh(proj['boundary'], name='boundary', 
           color='azure', render_lines_as_tubes=False, line_width=10)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4dc8

In [10]:
# Add the granitoid boundary
p.add_mesh(proj['top_granitoid'], name='top_granitoid',
           show_edges=False, style='wireframe')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4a08

In [11]:
# Add the granitoid boundary
#--
# Lets run a PVGeo filter to extract the topo surface from
#    the granitoid surface
ext = PVGeo.grids.ExtractTopography(
        remove=True, # remove the inactive cells
        tolerance=10.0 # buffer around the topo surface
        ).Apply(proj['top_granitoid'], topo)
granitoid = ext
#--
# Now plot it!
p.add_mesh(granitoid, name='top_granitoid',
           show_edges=False, style='surface',
           color='grey', opacity=0.75, reset_camera=False,
          )

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4fa8

In [12]:
# _ = p.screenshot('../integrated_view.png')

In [13]:
import pandas as pd
well_locs = pd.read_csv(gdc19.get_well_path('well_location_from_earth_model.csv'))
well_locs.head()

,Well ID,x,y,z (land surface),depth (m)
0,12-35,339385.010,4264212.985,1880.635342,2133.6
1,13-10,337709.018,4260659.997,1796.263259,1601.3
2,14-2,339393.002,4262171.961,1903.453009,1808.5
3,24-36,341221.016,4263743.011,2034.161472,1819.7
4,25-15,337873.021,4258706.959,1827.029208,2287.5


In [14]:
foo = PVGeo.pointsToPolyData(well_locs[['x', 'y', 'z (land surface)']].values)
p.add_mesh(foo, name='well_locations', point_size=10, color='orange')


(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x109dc4d68